In [1]:
# 1. Use a generator

# Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

# Let's define a generator and then run it as practice.

# **Answer the following questions:**

# - **Question 1: What is the sum of the outputs of the generator for limit = 5?**
# - **Question 2: What is the 13th number yielded**

# I suggest practicing these questions without GPT as the purpose is to further your learning.

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

# Question 1: What is the sum of the outputs of the generator for limit = 5?
sum_of_outputs = sum(generator)
print("Question 1 - Sum of outputs for limit = 5:", sum_of_outputs)

for sqrt_value in generator:
    print(sqrt_value)

# Question 2: What is the 13th number yielded
generator = square_root_generator(13)
thirteenth_number = None
for _ in range(13):
    thirteenth_number = next(generator)
print("Question 2 - 13th number yielded:", thirteenth_number)

Question 1 - Sum of outputs for limit = 5: 8.382332347441762
Question 2 - 13th number yielded: 3.605551275463989


In [30]:
# 2. Append a generator to a table with existing data

# Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

# 1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
# 2. Append the second generator to the same table as the first.
# 3. **After correctly appending the data, calculate the sum of all ages of people.**

# def people_1():
#     for i in range(1, 6):
#         yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# for person in people_1():
#     print(person)


# def people_2():
#     for i in range(3, 9):
#         yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# for person in people_2():
#     print(person)

# Solution

import duckdb

# Define the generators
def people_1():
    for i in range(1, 6):
        yield (i, f"Person_{i}", 25 + i, "City_A", None)
        # yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield (i, f"Person_{i}", 30 + i, "City_B", f"Job_{i}")
        # yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Step 1: Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
people_1_data = list(people_1())  # Store the generated values in a list
sum_of_ages_1 = sum(person[2] for person in people_1_data)
print("Sum of ages of all people from generator 1:", sum_of_ages_1)

# Step 2: Append the second generator to the same table as the first.
con = duckdb.connect(database=':memory:', read_only=False)

# Create a table
con.execute("CREATE TABLE people (ID INT, Name VARCHAR, Age INT, City VARCHAR, Occupation VARCHAR)")

# Insert data from the first generator
con.executemany("INSERT INTO people VALUES (?, ?, ?, ?, ?)", people_1_data)

# Append data from the second generator
con.executemany("INSERT INTO people VALUES (?, ?, ?, ?, ?)", people_2())

# Step 3: calculate the sum of all ages of people
result = con.execute("SELECT SUM(Age) FROM people")
sum_of_all_ages = result.fetchone()[0]
print("Sum of all ages of people after appending:", sum_of_all_ages)

Sum of ages of all people from generator 1: 140
Sum of all ages of people after appending: 353


In [28]:
# Question 3: Merge a generator

# Create a new table with primary key if it doesn't exist
con.execute("CREATE TABLE IF NOT EXISTS merged_people (ID INT PRIMARY KEY, Name VARCHAR, Age INT, City VARCHAR, Occupation VARCHAR)")

# Load data from the first generator
con.executemany("INSERT INTO merged_people VALUES (?, ?, ?, ?, NULL) ON CONFLICT(ID) DO UPDATE SET Name=excluded.Name, Age=excluded.Age, City=excluded.City", ((person[0], person[1], person[2], person[3]) for person in people_1()))

# Merge data from the second generator
con.executemany("INSERT INTO merged_people VALUES (?, ?, ?, ?, ?) ON CONFLICT(ID) DO UPDATE SET Name=excluded.Name, Age=excluded.Age, City=excluded.City, Occupation=excluded.Occupation", ((person[0], person[1], person[2], person[3], person[4]) for person in people_2()))

# Query to verify the loaded data
result = con.execute("SELECT * FROM merged_people")
print("Records after merging:")
while True:
    row = result.fetchone()
    if row is None:
        break
    print(row)

# Calculate the sum of ages of all the people loaded
result = con.execute("SELECT SUM(Age) FROM merged_people")
sum_of_merged_ages = result.fetchone()[0]
print("Sum of ages of all people after merging:", sum_of_merged_ages)



Records after merging:
(1, 'Person_1', 26, 'City_A', None)
(2, 'Person_2', 27, 'City_A', None)
(3, 'Person_3', 33, 'City_B', 'Job_3')
(4, 'Person_4', 34, 'City_B', 'Job_4')
(5, 'Person_5', 35, 'City_B', 'Job_5')
(6, 'Person_6', 36, 'City_B', 'Job_6')
(7, 'Person_7', 37, 'City_B', 'Job_7')
(8, 'Person_8', 38, 'City_B', 'Job_8')
Sum of ages of all people after merging: 266
